# MultiLayer Perceptron (MLP)

A Multilayer Perceptron (MLP), or Feedforward Neural Network, is a series of layers where each layer passes outputs to the next layer. It is useful for both regression and classification problems.

In this notebook, we will use the California Housing dataset to predict median house values.


## Setup and Load Data


In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Load California housing dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features to have mean 0 and variance 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## MLP using TensorFlow

In [ ]:
# Install TensorFlow if not already installed
# pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Build a feedforward neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # 1st hidden layer
    Dropout(0.2),  # Dropout helps prevent overfitting
    Dense(64, activation='relu'),  # 2nd hidden layer
    Dropout(0.2),
    Dense(1)  # Output layer for regression (1 output node)
])

# Compile the model with loss and optimizer
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Early stopping to stop training when validation loss stops improving
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2,
                    epochs=100, batch_size=32, callbacks=[early_stop])



Epoch 1/100


/usr/local/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - loss: 1.5380 - mae: 0.8590 - val_loss: 0.5171 - val_mae: 0.4952
Epoch 2/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - loss: 0.5853 - mae: 0.5367 - val_loss: 0.4319 - val_mae: 0.4662
Epoch 3/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - loss: 0.5072 - mae: 0.5010 - val_loss: 0.4209 - val_mae: 0.4454
Epoch 4/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - loss: 0.4636 - mae: 0.4863 - val_loss: 0.4050 - val_mae: 0.4501
Epoch 5/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 0.4569 - mae: 0.4775 - val_loss: 0.4368 - val_mae: 0.4441
Epoch 6/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - loss: 0.4311 - mae: 0.4683 - val_loss: 0.3891 - val_mae: 0.4477
Epoch 7/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - loss: 0.4053 - mae: 0.4585 - val_loss: 0.3774 - val_mae: 0.4396
Epoch 8/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 0.4045 - mae: 0.4505 - val_loss: 0.3737 - val_mae: 0.4279
Epoch 9/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [8]:
# Evaluate performance
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"TensorFlow Model - Test MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - loss: 0.2634 - mae: 0.3494
TensorFlow Model - Test MSE: 0.2693, MAE: 0.3504


## MLP using PyTorch

In [2]:
# Install PyTorch if not already installed
# pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Load data in batches
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)

# Define a simple MLP model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(X_train.shape[1], 64)  # 1st hidden layer
        self.hidden2 = nn.Linear(64, 64)                # 2nd hidden layer
        self.output = nn.Linear(64, 1)                  # Output layer
        self.dropout = nn.Dropout(0.2)            # Dropout probability
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x)) # Activation function for the first layer
        x = self.dropout(x) # Dropout for the first layer
        x = self.relu(self.hidden2(x)) # Activation function for the second layer
        x = self.dropout(x)
        return self.output(x)

model = MLP()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [4]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()                       # <-- turn on training mode
    for xb, yb in train_loader:         # iterate mini-batches
        optimizer.zero_grad()           # 1⃣  clear old gradients
        preds = model(xb)               # 2⃣  forward pass
        loss  = loss_fn(preds, yb)      # 3⃣  compute loss
        loss.backward()                 # 4⃣  back-propagate
        optimizer.step()                # 5⃣  update parameters


In [5]:

# Set model to evaluation mode
model.eval()

# Make predictions on the test set
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# Convert predictions and true values to NumPy arrays
y_pred = y_pred_tensor.numpy().flatten()
y_true = y_test_tensor.numpy().flatten()

# Compute metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

Test MSE: 0.3060
Test MAE: 0.3696
R² Score: 0.7665
